In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#credit for this code: https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/
#this code creates a function that gets the browser to scroll down
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

In [5]:
chrome_path = "/Users/yelenanevel/Downloads/chromedriver"
driver = webdriver.Chrome(executable_path = chrome_path)

final_df = pd.DataFrame(columns = ['name'])

page_num = 1 #the first page

while True:

    url = 'https://www.sephora.com/shop/clean-skin-care?pageSize=300&currentPage=' + str(page_num)
    driver.get(url)
    time.sleep(20)
    try: #check to see if the page is empty   
        if driver.find_element_by_class_name('css-3a7b4s').is_displayed():
            break

    except:

    #check to see if there is a pop up windew
        try:
            #exit the pop up window
            xpath = '//*[@id="modalDialog"]/button'
            btn = driver.find_element_by_xpath(xpath)
            btn.click()
            time.sleep(20)
        except:
            pass

        #as scrolling check if there is any more room to scroll
        old_len = 0
        while True:
            browser = scrollDown(driver, 20) #scroll down the page
            time.sleep(10) #give it time to load
            slug = driver.find_elements_by_class_name('css-ix8km1') #look for the urls of products
            new_len = len(slug)
            if old_len == new_len: #if the old length and new length are equal, the bottom of page was reached
                break
            else:
                old_len = new_len
        
        #now we will look for all the brands and product names
        html = driver.page_source
        soup = BeautifulSoup(html)
       
        
        #finding all the product names
        names = soup.find_all('span', {'class':"css-pelz90"})

          
    brands_names = []
    for i in range(len(names)):
        brand_name = {}
        brand_name['name'] = names[i].text
        
        brands_names.append(brand_name)
        
    

    #append our data frame with categories and URLs
    df = pd.DataFrame(brands_names)
    #print(df.head())
    #adding to go to next page
    page_num += 1
    #concatenating to get all in same df
    final_df = pd.concat([final_df, df], axis = 0, ignore_index = True) 

#close the chrome
driver.close()


final_df.to_csv('../data/clean_sephora_products.csv', index = False)

In [6]:
final_df.head()

,name
0,Protini™ Polypeptide Moisturizer
1,C-Firma™ Vitamin C Day Serum
2,The Water Cream
3,Yo Glow Facial Enzyme Scrub
4,The Dewy Skin Cream
